# Descripción Estadística de Casos Nuevos diarios, Fallecimientos acumulados y Ocupación de Camas UCI diarias por COVID para V Region
por Dr. Egidio Céspedes, Científico de Datos, SEREMI V Región

Datos          : https://github.com/MinCiencia/Datos-COVID19

Fuente Casos Chile   : Ministerio de Salud. Ver en: https://www.minsal.cl/nuevo-coronavirus-2019-ncov/casos-confirmados-en-chile-covid-19/

Fuente Casos Mundo   :  Our World in Data. Oxford Martin School, University of Oxford. Ver en: https://ourworldindata.org/coronavirus-source-data 

Fuente UCI Chile    : Reporte diario del Ministerio de Salud. Ver en: https://www.gob.cl/coronavirus/cifrasoficiales/#reportes

Fuente Muertos : Reporte diario del Ministerio de Salud. Ver en: https://www.gob.cl/coronavirus/cifrasoficiales/#reportes

Fuente Muertos Mundo   :  Our World in Data. Oxford Martin School, University of Oxford. Ver en: https://ourworldindata.org/coronavirus-source-data

Datos de Actualización Diaria

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

Se leen las BDs desde GitHub directamente

In [2]:
#Se leen las BDs chilenas desde GitHub directamente.
TxR = pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto3/TotalesPorRegion_T.csv")
UCI = pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto8/UCI_T.csv")
MxD = pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto14/FallecidosCumulativo_T.csv")

#Se leen las BDs del mundo desde GitHub directamente
NewCases=pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/jhu/new_cases.csv")
Deads=pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/jhu/new_deaths.csv")

Se modifican las BDs, se dejan solo datos necesarios y se dejan columnas en formato fecha correspondiente

In [3]:
def muertos(df):
    a=0
    lista=[]
    for i in df.Valparaíso:
        lista.append(i-a)
        a=i
    df['Valparaíso']=lista
    
    b=0
    lista2=[]
    for i in df.Total:
        lista2.append(i-b)
        b=i
    df['Total']=lista2

In [4]:
Casos=pd.concat([TxR[['Region']].drop(TxR[['Region']].index[[0]]),
                 TxR.iloc[:,19:35].drop(TxR.iloc[:,19:34].index[[0]]).apply(pd.to_numeric).apply(pd.to_numeric)], 
                axis=1)
Casos=Casos.rename(columns={'Region':'Fecha','Valparaíso.1':'Valparaíso','Total.1':'Total'})
Casos.Fecha=pd.to_datetime(Casos.Fecha)

MxD=MxD.rename(columns={'Region':'Fecha'})
MxD.Fecha=pd.to_datetime(MxD.Fecha)
muertos(MxD)

UCI=UCI.rename(columns={'Region':'Fecha'})
UCI=pd.concat([UCI.Fecha.drop(UCI.Fecha.index[[0,1]]),
               UCI.iloc[:,1:].drop(UCI.index[[0,1]]).apply(pd.to_numeric)],
              axis=1)
drop=UCI.drop(['Fecha'], axis=1)
UCI.Fecha=pd.to_datetime(UCI.Fecha)
UCI['Total']=drop.sum(axis=1)

Se crean las funciones que generan el reporte

In [5]:
def informe_nac(df,name):
    M=pd.DataFrame([df['Total'].mean(),
                    df['Total'].sum(),
                    df['Total'].mean().pct_change()])
    M=M.T
    M.columns = ['Prom_Sem_Nac'+name,'Total_Nac'+name,'Velocidad_Nac'+name]
    M.drop(M.tail(1).index,inplace=True)
    return M

def informe_valpo(df,name):
    M=pd.DataFrame([df['Valparaíso'].mean(),
                    df['Valparaíso'].sum(),
                    df['Valparaíso'].mean().pct_change()])
    M=M.T
    M.columns = ['Prom_Sem_Valp'+name,'Total_Valp'+name,'Velocidad_Valp'+name]
    M.drop(M.tail(1).index,inplace=True)
    return M

def informe_pais(df1,df2,name):
    df1.rename(columns={'date': 'Fecha'}, inplace=True)
    df1.Fecha=pd.to_datetime(NewCases.Fecha)
    agrupados1=df1.groupby([pd.Grouper(key='Fecha', freq='W-SUN')])
    M=pd.DataFrame([agrupados1[name].mean(),
                    agrupados1[name].sum(),
                    agrupados1[name].mean().pct_change()])
    M=M.T
    M.columns = ['Prom_Sem_Casos_'+name,'Total_Sem_Casos_'+name,'Velocidad_Sem_Casos_'+name]
    
    df2.rename(columns={'date': 'Fecha'}, inplace=True)
    df2.Fecha=pd.to_datetime(NewCases.Fecha)
    agrupados2=df2.groupby([pd.Grouper(key='Fecha', freq='W-SUN')])
    F=pd.DataFrame([agrupados2[name].mean(),
                    agrupados2[name].sum(),
                    agrupados2[name].mean().pct_change()])
    F=F.T
    F.columns = ['Prom_Sem_Muertes_'+name,'Total_Sem_Muertes_'+name,'Velocidad_Muertes_Casos_'+name]
    
    Z=pd.concat([M,F],axis=1)
    #Z.drop(Z.tail(1).index,inplace=True)
    Z=Z.tail(10)
    return Z

In [6]:
Casos2 = Casos.groupby([pd.Grouper(key='Fecha', freq='W-SUN')])
UCI2   = UCI.groupby([pd.Grouper(key='Fecha', freq='W-SUN')])
MxD2   = MxD.groupby([pd.Grouper(key='Fecha', freq='W-SUN')])

In [7]:
Valpo =pd.concat([informe_valpo(Casos2,'_Casos').tail(10),
           informe_valpo(UCI2,'_UCI').tail(10),
           informe_valpo(MxD2,'_Muertes').tail(10)], 
                 axis=1)

In [8]:
Chile=pd.concat([informe_nac(Casos2,'_Casos').tail(10),
           informe_nac(UCI2,'_UCI').tail(10),
           informe_nac(MxD2,'_Muertes').tail(10)],
                axis=1)

In [9]:
class Navegador():
    def __init__(self):
        print('**Inicio Modulo Navegador**\n')
        self.TS_inicial=dt.datetime.now()
        self.hoy=datetime.today().strftime('%d-%m-%Y').replace('-',"")
        self.nombre_archivo = None
    
    def TS():
    	t = dt.datetime.now().strftime('%m/%d %H:%M:%S')
    	evol = str((dt.datetime.now() - self.TS_inicial).seconds)
    	return str(t + '| [' + evol + 's]:')

    def custom_print(self, message_to_print, nombre=None):
    	# Le asigno el nombre de la fecha y hora si no tiene nombre previo
    	if nombre is None:
    		file_name ='LOG_'+datetime.today().strftime('%m-%d%H:%M:%S').replace('-',"")+ '.txt'
    	else:
    		file_name = str(nombre)

    	# Lo creo de cero si no existe.
    	# Si existe de antes.... lo apendo.
    	if self.nombre_archivo is None:
    		self.nombre_archivo = file_name
    		escribe = open(str(self.nombre_archivo), 'w')
    	else:
    		escribe = open(str(self.nombre_archivo), 'a')

    	escribe.write('\n' + message_to_print)
    	print(message_to_print)

In [10]:
list_semana = ['Semana -10', 'Semana -9', 'Semana -8', 'Semana -7', 'Semana -6', 'Semana -5', 'Semana -4', 'Semana -3', 'Semana -2', 'Semana -1' ]
Valpo['Semana comparacion'] = list_semana
Valpo = Valpo[['Semana comparacion', 'Prom_Sem_Valp_Casos', 'Total_Valp_Casos', 'Velocidad_Valp_Casos',
       'Prom_Sem_Valp_UCI', 'Total_Valp_UCI', 'Velocidad_Valp_UCI',
       'Prom_Sem_Valp_Muertes', 'Total_Valp_Muertes', 'Velocidad_Valp_Muertes',
       ]]

In [67]:
import datetime as dt
from datetime import datetime

dum = Valpo.to_latex(float_format="%0.1f", header = False, index_names = False)
dum = dum.split('\n')[slice(2,12)]

nv = Navegador()
nv.custom_print(
    ' '.join(dum),
    'Tabla1.tex')

**Inicio Modulo Navegador**

2022-05-15 &  Semana -10 &  414.1 &  2899.0 &  0.4 & 15.3 & 107.0 & -0.2 & 2.0 & 14.0 & -0.4 \\ 2022-05-22 &   Semana -9 &  626.7 &  4387.0 &  0.5 & 13.0 &  91.0 & -0.1 & 1.3 &  9.0 & -0.4 \\ 2022-05-29 &   Semana -8 &  870.3 &  6092.0 &  0.4 &  9.4 &  66.0 & -0.3 & 1.6 & 11.0 &  0.2 \\ 2022-06-05 &   Semana -7 &  966.7 &  6767.0 &  0.1 & 10.0 &  70.0 &  0.1 & 2.7 & 19.0 &  0.7 \\ 2022-06-12 &   Semana -6 & 1331.4 &  9320.0 &  0.4 &  9.7 &  68.0 & -0.0 & 2.9 & 20.0 &  0.1 \\ 2022-06-19 &   Semana -5 & 1525.7 & 10680.0 &  0.1 &  9.9 &  69.0 &  0.0 & 4.0 & 28.0 &  0.4 \\ 2022-06-26 &   Semana -4 & 1298.0 &  9086.0 & -0.1 & 10.0 &  70.0 &  0.0 & 3.0 & 21.0 & -0.2 \\ 2022-07-03 &   Semana -3 & 1151.9 &  8063.0 & -0.1 & 12.3 &  86.0 &  0.2 & 4.3 & 30.0 &  0.4 \\ 2022-07-10 &   Semana -2 &  948.1 &  6637.0 & -0.2 & 19.3 & 135.0 &  0.6 & 6.3 & 44.0 &  0.5 \\ 2022-07-17 &   Semana -1 &  755.7 &  5290.0 & -0.2 & 17.6 & 123.0 & -0.1 & 7.1 & 50.0 &  0.1 \\


In [ ]:
Chile

In [ ]:
informe_pais(NewCases,Deads,'Argentina')

In [ ]:
informe_pais(NewCases,Deads,'Peru')